## Primary & Blooming Model

In [1]:
from keras.models import Sequential, load_model, Model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16, preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.externals import joblib

import numpy as np
import cPickle as pickle
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
sys.path.insert(0, '..')
import paths
sys.path.insert(0, '../rainforest')
import data
import data_generators_leo as dgl

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 1050 Ti (0000:09:00.0)


### Load training data and labels.

In [7]:
train_csv = data.get_data()
train_labels = np.asarray(train_csv)[:,1:18]
train_data = np.asarray(train_csv)[:,0]
val_csv = data.get_data(train=False)
val_labels = np.asarray(val_csv)[:,1:18]
val_data = np.asarray(val_csv)[:,0]
image_size = 256
nb_train = len(train_data)
nb_val = 100

### Generators

In [8]:
train_gen = dgl.get_data(train_data, '../data/train-jpg', train_labels,
                                      flip=True, shift_x=5, shift_y=5, rot_range=10,
                                      elastic_trans=False, batch_size=8, balance_batches=True)

val_gen = dgl.get_data(val_data, '../data/train-jpg', val_labels, batch_size=8)



### Network Structure

In [9]:
model = VGG16(include_top=False, weights='imagenet', input_shape=(256,256,3))

for layer in model.layers:
    layer.trainable = False
    
layer = Flatten()(model.output)
layer = BatchNormalization()(layer)
layer = Dense(512, activation='relu')(layer)
layer = BatchNormalization()(layer)

output_layer = Dense(2, activation='softmax')(layer)

### Training Network

In [12]:
model = Model(input=model.input, output=output_layer)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.fit_generator(train_gen, samples_per_epoch=100, nb_epoch=20, verbose=1,
                    validation_data=val_gen, nb_val_samples=100)

/home/pieter/anaconda2/envs/mlippython2/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=/input_2)`
  """Entry point for launching an IPython kernel.
/home/pieter/anaconda2/envs/mlippython2/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
  
/home/pieter/anaconda2/envs/mlippython2/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=1, validation_data=<generator..., steps_per_epoch=100, epochs=20, validation_steps=100)`
  
INFO (theano.gof.compilelock): Refreshing lock /home/pieter/.theano/compiledir_Linux-4.9--amd64-x86_64-with-debian-9

Epoch 1/20


ValueError: Error when checking model input: expected input_2 to have shape (None, 256, 256, 3) but got array with shape (8, 3, 256, 256)